# **Credit Collections Behavior Model: PTP Funnel, Risk Segments, and Recovery Cohorts**

Structured queries powering cohort analysis and PTP conversion modeling across risk bands and outreach channels.

## **1\. Accounts by Risk Band**

### How many accounts fall into each credit risk tier (A–E)

In [39]:
SELECT risk_band, COUNT(*) AS num_accounts
FROM dbo.accounts
GROUP BY risk_band
ORDER BY risk_band;

(5 rows affected)

Total execution time: 00:00:01.141

risk_band,num_accounts
A,21993
B,26082
C,24028
D,17900
E,9997


## **2\. Credit Limit Stats (Min, Max, Avg, Median)**

### Profile the overall credit limit distribution

In [38]:
SELECT DISTINCT
  MIN(credit_limit) OVER () AS min_limit,
  MAX(credit_limit) OVER () AS max_limit,
  AVG(credit_limit) OVER () AS avg_limit,
  PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY credit_limit) OVER () AS median_limit
FROM dbo.accounts;

: Msg 8117, Level 16, State 1, Line 4
Operand data type nvarchar(max) is invalid for avg operator.

Total execution time: 00:00:00.062

## **3\. Accounts by Country and Region**

### Geographic distribution of accounts.

In [6]:
SELECT country, region, COUNT(*) AS total
FROM dbo.accounts
GROUP BY country, region
ORDER BY total DESC;

(26 rows affected)

Total execution time: 00:00:13.438

country,region,total
CA,NS,4699
CA,SK,4655
CA,MB,4648
CA,PE,4648
CA,QC,4629
CA,BC,4617
CA,YT,4610
CA,AB,4594
CA,NL,4576
CA,NU,4574


## **4\. DPD Bucket by Month**

### Monthly delinquency profile by how late customers are

In [19]:
SELECT
  DATETRUNC(MONTH, CAST(snapshot_month AS DATE)) AS month,
  dpd_bucket,
  COUNT(*) AS accounts
FROM dbo.deliqunecy
GROUP BY DATETRUNC(MONTH, CAST(snapshot_month AS DATE)) , dpd_bucket
ORDER BY month, dpd_bucket DESC;

(40 rows affected)

Total execution time: 00:00:16.215

month,dpd_bucket,accounts
2025-01-01,90,3473
2025-01-01,60,4605
2025-01-01,30,9140
2025-01-01,120,1960
2025-01-01,0,80822
2025-02-01,90,4926
2025-02-01,60,5921
2025-02-01,30,10138
2025-02-01,120,5355
2025-02-01,0,73660


## **5\. DPD by Risk Band**

### Compare late-payment severity across risk levels.

In [20]:
SELECT
  a.risk_band,
  d.dpd_bucket,
  COUNT(*) AS accounts
FROM dbo.deliqunecy d
JOIN dbo.accounts a ON d.account_id = a.account_id
GROUP BY a.risk_band, d.dpd_bucket
ORDER BY a.risk_band, d.dpd_bucket;

(25 rows affected)

Total execution time: 00:01:06.718

risk_band,dpd_bucket,accounts
A,0,144194
A,120,3613
A,30,15730
A,60,7263
A,90,5144
B,0,155088
B,120,8837
B,30,22194
B,60,12329
B,90,10208


## **6\. Outreach Outcome Counts by Channel**

### how many contact attempts were made and what happened.

In [21]:
SELECT
  channel,
  outcome,
  COUNT(*) AS attempts
FROM dbo.outreach
GROUP BY channel, outcome
ORDER BY channel, outcome;

(12 rows affected)

Total execution time: 00:00:24.625

channel,outcome,attempts
AgentCall,LeftVM,10228
AgentCall,NoAnswer,18306
AgentCall,PTP,25783
AgentCall,Reached,48322
Email,LeftVM,20351
Email,NoAnswer,87818
Email,PTP,20767
Email,Reached,75712
SMS,LeftVM,26267
SMS,NoAnswer,86289


## **7\. PTP Rate by Channel**

### What % of contacts resulted in a “Promise to Pay” (PTP), by channel.

In [22]:
SELECT
  channel,
  COUNT(*) AS total_attempts,
  SUM(CASE WHEN outcome = 'PTP' THEN 1 ELSE 0 END) AS ptp_count,
  1.0 * SUM(CASE WHEN outcome = 'PTP' THEN 1 ELSE 0 END) / COUNT(*) AS ptp_rate
FROM dbo.outreach
GROUP BY channel
ORDER BY ptp_rate DESC;

(3 rows affected)

Total execution time: 00:00:09.270

channel,total_attempts,ptp_count,ptp_rate
AgentCall,102639,25783,0.251200810608
SMS,262286,39193,0.149428486461
Email,204648,20767,0.101476681912


## **8\. Kept vs Broken Promises**

### How many PTPs were honored versus broken.

In [23]:
SELECT
  kept_promise,
  COUNT(*) AS num_accounts
FROM dbo.followup
GROUP BY kept_promise;

(2 rows affected)

Total execution time: 00:00:05.220

kept_promise,num_accounts
True,52415
False,33328


## **9\. PTP Kept Rate by Channel**

### how effective each contact method is at generating real payments.

In [27]:
SELECT
  o.channel,
  COUNT(*) AS ptp_total,
  SUM(CASE WHEN f.kept_promise = 'True' THEN 1 ELSE 0 END) AS ptp_kept,
  1.0 * SUM(CASE WHEN f.kept_promise = 'True' THEN 1 ELSE 0 END) / COUNT(*) AS ptp_kept_rate
FROM dbo.followup f
JOIN dbo.outreach o ON f.contact_id = o.contact_id
GROUP BY o.channel
ORDER BY ptp_kept_rate DESC;

(3 rows affected)

Total execution time: 00:00:20.956

channel,ptp_total,ptp_kept,ptp_kept_rate
Email,20767,12728,0.612895459141
SMS,39193,23973,0.611665348404
AgentCall,25783,15714,0.609471357095


## **10\. Top 10 Largest Payments**

### Spot the biggest individual payments made.

In [28]:
SELECT TOP 10 *
FROM dbo.payments
ORDER BY amount DESC;

(10 rows affected)

Total execution time: 00:00:01.996

payment_id,account_id,payment_date,amount,method
3007944,72002,2025-04-08,999.99,Card
7005581,31806,2025-08-07,999.99,ACH
4001274,46114,2025-05-13,999.95,Cash
3003890,57542,2025-04-07,999.94,ACH
4001560,4529,2025-05-26,999.93,Cash
7004910,12673,2025-08-07,999.89,Card
4002562,43284,2025-05-01,999.88,Card
7005386,14370,2025-08-28,999.85,ACH
2001184,40962,2025-03-22,999.81,ACH
6002684,396,2025-07-01,999.79,Card


## **11\. Payment Method Usage (Fixed Type Casting)**

### See volume and value per payment method.

In [30]:
SELECT
  method,
  COUNT(*) AS count,
  SUM(CAST(amount AS DECIMAL(12,2))) AS total_amount,
  AVG(CAST(amount AS DECIMAL(12,2))) AS avg_amount
FROM dbo.payments
GROUP BY method
ORDER BY total_amount DESC;

(3 rows affected)

Total execution time: 00:00:07.357

method,count,total_amount,avg_amount
ACH,57189,74324497.36,1299.629253
Card,34817,45521942.04,1307.463079
Cash,10426,13794430.62,1323.079859


## **12\. Avg Payment by Risk Band (Fixed)**

### Are higher-risk accounts paying less?

In [32]:
SELECT
  a.risk_band,
  COUNT(p.payment_id) AS num_payments,
  AVG(CAST(p.amount AS DECIMAL(12,2))) AS avg_payment,
  SUM(CAST(p.amount AS DECIMAL(12,2))) AS total_payment
FROM dbo.payments p
JOIN dbo.accounts a ON p.account_id = a.account_id
GROUP BY a.risk_band
ORDER BY a.risk_band;

(5 rows affected)

Total execution time: 00:00:19.778

risk_band,num_payments,avg_payment,total_payment
A,11439,1084.764053,12408616.01
B,19397,1171.035221,22714570.19
C,26705,1290.292736,34457267.54
D,26791,1407.084521,37697201.41
E,18100,1456.531208,26363214.87


## **13\. 30 → 60 Day Roll Rates**

### What % of 30-day late accounts rolled to 60 days next month.

In [40]:
WITH d AS (
  SELECT account_id, CAST(snapshot_month AS DATE) AS snapshot_month, dpd_bucket
  FROM dbo.deliqunecy
),
next_dpd AS (
  SELECT
    d1.account_id,
    DATETRUNC(MONTH, d1.snapshot_month) AS month,
    d1.dpd_bucket AS dpd_start,
    d2.dpd_bucket AS dpd_next
  FROM d d1
  JOIN d d2
    ON d1.account_id = d2.account_id
    AND DATEADD(MONTH, 1, d1.snapshot_month) = d2.snapshot_month
  WHERE d1.dpd_bucket = 30
)
SELECT
  month,
  COUNT(*) AS was_30,
  SUM(CASE WHEN dpd_next = 60 THEN 1 ELSE 0 END) AS moved_60,
  1.0 * SUM(CASE WHEN dpd_next = 60 THEN 1 ELSE 0 END) / COUNT(*) AS roll_rate_30_to_60
FROM next_dpd
GROUP BY month
ORDER BY month;

(7 rows affected)

Total execution time: 00:00:22.084

month,was_30,moved_60,roll_rate_30_to_60
2025-01-01,9140,1370,0.149890590809
2025-02-01,10138,1447,0.142730321562
2025-03-01,10758,1458,0.135527049637
2025-04-01,10915,1442,0.132111772789
2025-05-01,11088,1459,0.131583694083
2025-06-01,11207,1500,0.133844918354
2025-07-01,11508,1550,0.134688912061


## **14\. Days Between Outreach and Payment**

### Time lag between contact attempt and payment.

In [34]:
SELECT TOP 100
  o.account_id,
  o.contact_date,
  p.payment_date,
  DATEDIFF(DAY, o.contact_date, p.payment_date) AS days_to_pay,
  p.amount
FROM dbo.outreach o
JOIN dbo.payments p ON o.account_id = p.account_id
WHERE p.payment_date >= o.contact_date
ORDER BY days_to_pay ASC;

(100 rows affected)

Total execution time: 00:00:37.527

account_id,contact_date,payment_date,days_to_pay,amount
764,2025-01-27,2025-01-27,0,193.13
946,2025-01-16,2025-01-16,0,731.87
1203,2025-01-15,2025-01-15,0,1588.37
1279,2025-01-20,2025-01-20,0,1555.97
1306,2025-01-03,2025-01-03,0,610.59
1825,2025-01-25,2025-01-25,0,674.39
1864,2025-01-08,2025-01-08,0,314.12
1872,2025-01-26,2025-01-26,0,790.88
2363,2025-01-09,2025-01-09,0,188.36
2595,2025-01-29,2025-01-29,0,990.1


## **15\. Accounts With Multiple Broken PTPs**

### Accounts that repeatedly promised to pay but didn’t.

In [36]:
SELECT account_id, COUNT(*) AS broken_promises
FROM dbo.followup
WHERE kept_promise = 'False'
GROUP BY account_id
HAVING COUNT(*) > 1
ORDER BY broken_promises DESC;

(6691 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:04.864

account_id,broken_promises
33677,6
81908,6
83900,6
83986,6
85441,5
90170,5
91662,5
96980,5
97282,5
99735,5


## **16\. Creating Final View for Dashboard**

In [42]:
CREATE OR ALTER VIEW vw_collections_dashboard AS
SELECT
  -- Account info
  a.account_id,
  ISNULL(a.risk_band, 'Unknown') AS risk_band,
  ISNULL(a.credit_limit, 0.00) AS credit_limit,
  ISNULL(a.region, 'Unknown') AS region,
  ISNULL(a.country, 'Unknown') AS country,

  -- Delinquency snapshot
  CAST(d.snapshot_month AS DATE) AS snapshot_month,
  CONVERT(VARCHAR(7), CAST(d.snapshot_month AS DATE), 120) AS snapshot_month_str,
  ISNULL(d.dpd_bucket, 0) AS dpd_bucket,

  -- Derived delinquency flags
  CASE WHEN d.dpd_bucket > 0 THEN 1 ELSE 0 END AS is_delinquent,
  CASE d.dpd_bucket
    WHEN 0 THEN 0
    WHEN 30 THEN 1
    WHEN 60 THEN 2
    WHEN 90 THEN 3
    WHEN 120 THEN 4
    ELSE NULL
  END AS dpd_severity_score,

  -- Payments (aggregated)
  ISNULL(p.last_payment_date, NULL) AS last_payment_date,
  ISNULL(p.total_payments, 0.00) AS total_payments,
  ISNULL(p.avg_payment, 0.00) AS avg_payment,

  -- Outreach
  ISNULL(o.contact_date, NULL) AS contact_date,
  ISNULL(o.channel, 'None') AS channel,
  ISNULL(o.outcome, 'None') AS outcome,
  CASE WHEN o.outcome = 'PTP' THEN 1 ELSE 0 END AS has_ptp,

  -- PTP follow-up
  ISNULL(f.ptp_payment_due, NULL) AS ptp_payment_due,
  ISNULL(f.kept_promise, 'None') AS kept_promise,
  CASE 
    WHEN f.kept_promise = 'True' THEN 1
    WHEN f.kept_promise = 'False' THEN 0
    ELSE NULL
  END AS ptp_kept_score,
  CASE 
    WHEN f.kept_promise = 'True' THEN 'Kept'
    WHEN f.kept_promise = 'False' THEN 'Broken'
    ELSE 'None'
  END AS ptp_outcome,

  -- Channel rank (for funnel sorting)
  CASE o.channel
    WHEN 'SMS' THEN 1
    WHEN 'Email' THEN 2
    WHEN 'AgentCall' THEN 3
    ELSE 4
  END AS last_contact_channel_rank

FROM dbo.deliqunecy d

-- Join to accounts
JOIN dbo.accounts a ON d.account_id = a.account_id

-- Join aggregated payments
LEFT JOIN (
  SELECT
    account_id,
    MAX(payment_date) AS last_payment_date,
    SUM(CAST(amount AS DECIMAL(12,2))) AS total_payments,
    AVG(CAST(amount AS DECIMAL(12,2))) AS avg_payment
  FROM dbo.payments
  GROUP BY account_id
) p ON d.account_id = p.account_id

-- Join latest outreach
LEFT JOIN (
  SELECT
    account_id,
    contact_date,
    channel,
    outcome,
    contact_id,
    ROW_NUMBER() OVER (PARTITION BY account_id ORDER BY contact_date DESC) AS rn
  FROM dbo.outreach
) o ON d.account_id = o.account_id AND o.rn = 1

-- Join PTP follow-up
LEFT JOIN dbo.followup f ON o.contact_id = f.contact_id;

Commands completed successfully.

Total execution time: 00:08:18.542